<a href="https://colab.research.google.com/github/rodolfolermacontreras/Fine-Tuning-For-Sentiment-Analysis/blob/main/Finetuning_BERT_Classifications.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Importing the libraries needed
import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer

In [4]:
!pip install transformers datasets

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
from transformers import DistilBertTokenizer, DistilBertModel, AdamW, get_linear_schedule_with_warmup
from datasets import Dataset
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.optim import AdamW
from tqdm import tqdm
from sklearn.metrics import classification_report

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [126]:
# Load the dataset
df = pd.read_csv('Fine_Tune_Data.csv')

In [127]:
df.head()

,img_id,Multiple Faces Score,Title,Body,Update,Original Index,split_index,data_row.global_key,data_row.row_data,In-calls Only or Not,...,Title_Clean_Basics,Body_HTML,Title_HTML,Body_Clean,Title_Clean,Clean_Text,Clean_Text_Basic,All_Emojis,Weak_Label,Label
0,124959181,0.51,Start KittenAn Unforgettable Companion with Cl...,\n+An Unforgettable Companion with Class & Int...,0,8711,0,0,Title: \n\nStart KittenAn Unforgettable Compan...,No,...,start kittenan unforgettable companion with cl...,\n+An Unforgettable Companion with Class & Int...,Start KittenAn Unforgettable Companion with Cl...,+an unforgettable companion class & intelligen...,start kittenan unforgettable companion class &...,start kittenan unforgettable companion class &...,start kittenan unforgettable companion with cl...,\n\n,0,0
1,124925419,0.51,Mya whiteI am the right amount of spice to add...,\n+I am the right amount of spice to add to yo...,0,5912,2,2,Title: \n\nMya whiteI am the right amount of s...,No,...,mya whitei am the right amount of spice to add...,\n+I am the right amount of spice to add to yo...,Mya whiteI am the right amount of spice to add...,+i right amount spice add life save profile vi...,mya whitei right amount spice add life,mya whitei right amount spice add life\n\n+i r...,mya whitei am the right amount of spice to add...,\n\n,0,0
2,124908462,0.51,IsaGemLet’s have fun in Miami together,\n+Let’s have fun in Miami together \n Save th...,0,3807,4,4,Title: \n\nIsaGemLet’s have fun in Miami toget...,No,...,isagemlet’s have fun in miami together,\n+Let’s have fun in Miami together \n Save th...,IsaGemLet’s have fun in Miami together,+let ’ fun miami together save profile home ci...,isagemlet ’ fun miami together,isagemlet ’ fun miami together\n\n+let ’ fun m...,isagemlet’s have fun in miami together\n\n\n+l...,\n\n,0,0
3,124959959,0.80,Ms Eden winterUltimate Femdom Goddess,\n+Ultimate Femdom Goddess \n Save this profil...,0,9018,8,8,Title: \n\nMs Eden winterUltimate Femdom Godde...,No,...,ms eden winterultimate femdom goddess,\n+Ultimate Femdom Goddess \n Save this profil...,Ms Eden winterUltimate Femdom Goddess,+ultimate femdom goddess save profile home cit...,m eden winterultimate femdom goddess,m eden winterultimate femdom goddess\n\n+ultim...,ms eden winterultimate femdom goddess\n\n\n+ul...,\n\n,0,0
4,124959868,0.51,"JaneWhen we’re together, there’s nothing else.","\n+When we’re together, there’s nothing else. ...",0,8932,9,9,"Title: \n\nJaneWhen we’re together, there’s no...",No,...,"janewhen we’re together, there’s nothing else.","\n+When we’re together, there’s nothing else. ...","JaneWhen we’re together, there’s nothing else.","+when ’ together , ’ nothing else . save profi...","janewhen ’ together , ’ nothing else .","janewhen ’ together , ’ nothing else .\n\n+whe...","janewhen we’re together, there’s nothing else....",\n\n,0,0


In [128]:
# Preprocess the data
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [129]:
def tokenize(batch):
    texts = []
    labels = []
    for i in range(len(batch['text'])):
        if batch['label'][i] is not None:
            texts.append(str(batch['text'][i]))
            labels.append(int(batch['label'][i]))

    tokenized = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    return {'input_ids': tokenized['input_ids'].squeeze(),
            'attention_mask': tokenized['attention_mask'].squeeze(),
            'label': torch.tensor(labels)}

In [130]:
df.rename(columns={
    'Clean_Text': 'text',
    'Label': 'label'
}, inplace=True)

# Drop all other columns except 'text' and 'label'
df = df[['text', 'label']]

# Preprocess the data
df['text'] = df['text'].astype(str)
df['label'] = df['label'].astype(int)

<ipython-input-130-1b1ffa22b839>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].astype(str)
<ipython-input-130-1b1ffa22b839>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['label'].astype(int)


In [139]:
df.head()

,text,label
0,start kittenan unforgettable companion class &...,0
1,mya whitei right amount spice add life\n\n+i r...,0
2,isagemlet ’ fun miami together\n\n+let ’ fun m...,0
3,m eden winterultimate femdom goddess\n\n+ultim...,0
4,"janewhen ’ together , ’ nothing else .\n\n+whe...",0


In [131]:
dataset = Dataset.from_pandas(df)
dataset = dataset.map(tokenize, batched=True, remove_columns=['text', 'label'])

Map:   0%|          | 0/1849 [00:00<?, ? examples/s]

In [132]:
dataset

Dataset({
    features: ['label', 'input_ids', 'attention_mask'],
    num_rows: 1849
})

In [133]:
# Split the dataset into train and test sets
dataset = dataset.train_test_split(test_size=0.2)

In [134]:
# Define the model architecture
class DistilBERTClassifier(torch.nn.Module):
    def __init__(self, num_classes):
        super(DistilBERTClassifier, self).__init__()
        self.distilbert = DistilBertModel.from_pretrained('distilbert-base-uncased')
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]
        pooled_output = self.dropout(pooled_output)
        logits = self.linear(pooled_output)
        return logits

# Set up the training environment
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DistilBERTClassifier(num_classes=len(df['label'].unique())).to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)

In [135]:
# Create data loaders
train_loader = DataLoader(dataset['train'], batch_size=16, sampler=RandomSampler(dataset['train']))
test_loader = DataLoader(dataset['test'], batch_size=64, sampler=SequentialSampler(dataset['test']))

In [136]:
train_loader = list(train_loader)
test_loader = list(test_loader)

In [146]:
# # Fine-tune the model
# num_epochs = 3
# for epoch in range(num_epochs):
#     model.train()
#     for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['label'].to(device)

#         optimizer.zero_grad()
#         logits = model(input_ids, attention_mask)
#         loss = torch.nn.functional.cross_entropy(logits, labels)
#         loss.backward()
#         optimizer.step()

#     model.eval()
#     predictions = []
#     true_labels = []
#     for batch in tqdm(test_loader, desc="Evaluation"):
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['label'].to(device)

#         with torch.no_grad():
#             logits = model(input_ids, attention_mask)

#         predictions.extend(logits.argmax(dim=-1).tolist())
#         true_labels.extend(labels.tolist())

#     print(f"Epoch {epoch+1} Classification Report:")
#     print(classification_report(true_labels, predictions))

In [7]:
# Importing the libraries needed
import pandas as pd
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertModel, DistilBertTokenizer

!pip install transformers datasets

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
from transformers import DistilBertTokenizer, DistilBertModel, AdamW, get_linear_schedule_with_warmup
from datasets import Dataset
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.optim import AdamW
from tqdm import tqdm
from sklearn.metrics import classification_report

import torch
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from datasets import load_dataset
import numpy as np
from tqdm.auto import tqdm

# Load the dataset
df = pd.read_csv('Fine_Tune_Data.csv')

df = df.rename(columns={'Clean_Text': 'text', 'Label': 'label'})
df = df[['text', 'label']]
df['text'] = df['text'].astype(str)
df['label'] = df['label'].astype(int)

# Tokenization
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        inputs = self.tokenizer(text, padding='max_length', truncation=True, max_length=128, return_tensors="pt")
        return {
            'input_ids': inputs['input_ids'].squeeze(0),  # Remove batch dimension
            'attention_mask': inputs['attention_mask'].squeeze(0),  # Remove batch dimension
            'labels': torch.tensor(label)
        }

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Create datasets
train_dataset = CustomDataset(X_train.tolist(), y_train.tolist(), tokenizer)
test_dataset = CustomDataset(X_test.tolist(), y_test.tolist(), tokenizer)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Check if a GPU is available and set PyTorch to use the GPU, otherwise it will use the CPU
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Define the model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
model.to(device)

# # Optimizer
# optimizer = AdamW(model.parameters(), lr=5e-5)

# # Training loop
# num_epochs = 15
# for epoch in range(num_epochs):
#     model.train()
#     train_loss = 0
#     for batch in tqdm(train_loader, desc=f"Training Epoch {epoch+1}"):
#         batch = {k: v.to(device) for k, v in batch.items()}

#         outputs = model(**batch)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()
#         optimizer.zero_grad()

#         train_loss += loss.item()

#     print(f"Epoch {epoch+1} | Train Loss: {train_loss / len(train_loader)}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [8]:
from transformers import get_linear_schedule_with_warmup

# Training loop
num_epochs = 3
num_warmup_steps = 0
num_training_steps = num_epochs * len(train_loader)

# Apply weight decay for regularization
optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)

# Apply learning rate scheduling
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0

    for batch_idx, batch in enumerate(tqdm(train_loader, desc=f"Training Epoch {epoch+1}")):
        batch = {k: v.to(device) for k, v in batch.items()}

        # Apply gradient accumulation
        outputs = model(**batch)
        loss = outputs.loss / 4  # Accumulate gradients over 4 batches
        loss.backward()

        train_loss += loss.item()

        # Update weights and apply learning rate scheduling
        if (batch_idx + 1) % 4 == 0 or batch_idx == len(train_loader) - 1:  # Update weights after every 4 batches or at the end of the epoch
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

    print(f"Epoch {epoch+1} | Train Loss: {train_loss / len(train_loader)}")

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training Epoch 1:   0%|          | 0/185 [00:00<?, ?it/s]

Epoch 1 | Train Loss: 0.10991956333856324


Training Epoch 2:   0%|          | 0/185 [00:00<?, ?it/s]

Epoch 2 | Train Loss: 0.09931709906841452


Training Epoch 3:   0%|          | 0/185 [00:00<?, ?it/s]

Epoch 3 | Train Loss: 0.0818777243858455


In [9]:
# Evaluation
model.eval()
predictions = []
true_labels = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating"):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)
        predictions.extend(preds.cpu().numpy())
        true_labels.extend(batch['labels'].cpu().numpy())

# Classification report
print(classification_report(true_labels, predictions))

Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.89      0.89      0.89       293
           1       0.60      0.60      0.60        77

    accuracy                           0.83       370
   macro avg       0.75      0.75      0.75       370
weighted avg       0.83      0.83      0.83       370



In [10]:
# Save the trained model
model_save_path = 'distilbert_model.pth'
torch.save(model.state_dict(), model_save_path)

# Load the saved model
loaded_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
loaded_model.load_state_dict(torch.load(model_save_path))
loaded_model.to(device)
loaded_model.eval()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [11]:
# Make predictions on the test data
predictions = []
true_labels = []
with torch.no_grad():
    for batch in tqdm(test_loader, desc="Evaluating"):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = loaded_model(**batch)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)
        predictions.extend(preds.cpu().numpy())
        true_labels.extend(batch['labels'].cpu().numpy())

# Classification report
print("Performance on Test Data:")
print(classification_report(true_labels, predictions))

Evaluating:   0%|          | 0/12 [00:00<?, ?it/s]

Performance on Test Data:
              precision    recall  f1-score   support

           0       0.89      0.89      0.89       293
           1       0.60      0.60      0.60        77

    accuracy                           0.83       370
   macro avg       0.75      0.75      0.75       370
weighted avg       0.83      0.83      0.83       370

